In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
song = pd.read_csv("E:\\Minor\\project\\song_data.csv")
song.head()

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [3]:
interaction = pd.read_csv("E:\\Minor\\project\\triplets_file.csv")
interaction.head()

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [4]:
# combine both data
df = pd.merge(interaction, song.drop_duplicates(['song_id']), on='song_id', how='left')
df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [5]:
# creating new feature combining title and artist name
df['song'] = df['title']+' - '+ df['artist_name']
df.head()

,user_id,song_id,listen_count,title,release,artist_name,year,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0,The Cove - Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005,Constellations - Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999,Learn To Fly - Foo Fighters


In [6]:
df.shape

(2000000, 8)

# Popularity based Recommender system

In [7]:
# taking top 10k samples for quick results
#df = df.head(10000)

In [8]:
# cummulative sum of listen count of the songs
song_grouped = df.groupby(['song']).agg({'listen_count':'count'}).reset_index()
song_grouped.head()

,song,listen_count
0,#!*@ You Tonight [Featuring R. Kelly] (Explici...,78
1,#40 - DAVE MATTHEWS BAND,338
2,& Down - Boys Noize,373
3,' Cello Song - Nick Drake,103
4,'97 Bonnie & Clyde - Eminem,93


In [9]:
grouped_sum = song_grouped['listen_count'].sum()
song_grouped['percentage'] = (song_grouped['listen_count'] / grouped_sum ) * 100
song_grouped.sort_values(['listen_count', 'song'], ascending=[0,1])

,song,listen_count,percentage
7127,Sehr kosmisch - Harmonia,8277,0.41385
9084,Undo - Björk,7032,0.35160
2068,Dog Days Are Over (Radio Edit) - Florence + Th...,6949,0.34745
9880,You're The One - Dwight Yoakam,6412,0.32060
6774,Revelry - Kings Of Leon,6145,0.30725
...,...,...,...
3526,Historia Del Portero - Ricardo Arjona,51,0.00255
7071,Scared - Three Days Grace,51,0.00255
2147,Don´t Leave Me Now - Amparanoia,50,0.00250
2991,Ghosts (Toxic Avenger Mix) - Ladytron,48,0.00240


In [10]:
top_song = song_grouped.sort_values(['listen_count', 'song'], ascending=[0,1]).head(10)
top_song = top_song[['song', 'listen_count']].reset_index(drop=True)
print(top_song)

                                                song  listen_count
0                           Sehr kosmisch - Harmonia          8277
1                                       Undo - Björk          7032
2  Dog Days Are Over (Radio Edit) - Florence + Th...          6949
3                     You're The One - Dwight Yoakam          6412
4                            Revelry - Kings Of Leon          6145
5                              Secrets - OneRepublic          5841
6  Horn Concerto No. 4 in E flat K495: II. Romanc...          5385
7                     Fireflies - Charttraxx Karaoke          4795
8                           Hey_ Soul Sister - Train          4758
9                                 Tive Sim - Cartola          4548


In [11]:
#Class for Popularity based Recommender System model
class popularity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.popularity_recommendations = None
        
    #Create the popularity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

        #Get a count of user_ids for each unique song as recommendation score
        train_data_grouped = train_data.groupby([self.item_id]).agg({self.user_id: 'count'}).reset_index()
        train_data_grouped.rename(columns = {'user_id': 'score'},inplace=True)
    
        #Sort the songs based upon recommendation score
        train_data_sort = train_data_grouped.sort_values(['score', self.item_id], ascending = [0,1])
    
        #Generate a recommendation rank based upon score
        train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
        
        #Get the top 10 recommendations
        self.popularity_recommendations = train_data_sort.head(10)

    #Use the popularity based recommender system model to make recommendations
    def recommend(self, user_id):    
        user_recommendations = self.popularity_recommendations
        
        #Add user_id column for which the recommendations are being generated
        user_recommendations['user_id'] = user_id
    
        #Bring user_id column to the front
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols]
        
        return user_recommendations
    

In [12]:
pr = popularity_recommender_py()

In [13]:
pr.create(df, 'user_id', 'song')

In [14]:
# display the top 10 popular songs
pr.recommend(df['user_id'][5])

,user_id,song,score,Rank
7127,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Sehr kosmisch - Harmonia,8277,1.0
9084,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Undo - Björk,7032,2.0
2068,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Dog Days Are Over (Radio Edit) - Florence + Th...,6949,3.0
9880,b80344d063b5ccb3212f76538f3d9e43d87dca9e,You're The One - Dwight Yoakam,6412,4.0
6774,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Revelry - Kings Of Leon,6145,5.0
7115,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Secrets - OneRepublic,5841,6.0
3613,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Horn Concerto No. 4 in E flat K495: II. Romanc...,5385,7.0
2717,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Fireflies - Charttraxx Karaoke,4795,8.0
3485,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Hey_ Soul Sister - Train,4758,9.0
8847,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Tive Sim - Cartola,4548,10.0


In [15]:
pr.recommend(df['user_id'][111])

,user_id,song,score,Rank
7127,e006b1a48f466bf59feefed32bec6494495a4436,Sehr kosmisch - Harmonia,8277,1.0
9084,e006b1a48f466bf59feefed32bec6494495a4436,Undo - Björk,7032,2.0
2068,e006b1a48f466bf59feefed32bec6494495a4436,Dog Days Are Over (Radio Edit) - Florence + Th...,6949,3.0
9880,e006b1a48f466bf59feefed32bec6494495a4436,You're The One - Dwight Yoakam,6412,4.0
6774,e006b1a48f466bf59feefed32bec6494495a4436,Revelry - Kings Of Leon,6145,5.0
7115,e006b1a48f466bf59feefed32bec6494495a4436,Secrets - OneRepublic,5841,6.0
3613,e006b1a48f466bf59feefed32bec6494495a4436,Horn Concerto No. 4 in E flat K495: II. Romanc...,5385,7.0
2717,e006b1a48f466bf59feefed32bec6494495a4436,Fireflies - Charttraxx Karaoke,4795,8.0
3485,e006b1a48f466bf59feefed32bec6494495a4436,Hey_ Soul Sister - Train,4758,9.0
8847,e006b1a48f466bf59feefed32bec6494495a4436,Tive Sim - Cartola,4548,10.0


# Collaborative (Item Similarity) based Recommender system

In [16]:
#Class for Collaborative/Item similarity based Recommender System model
class item_similarity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.cooccurence_matrix = None
        self.songs_dict = None
        self.rev_songs_dict = None
        self.item_similarity_recommendations = None
        
    #Get unique items (songs) corresponding to a given user    
    def get_user_items(self, user):
        user_data = self.train_data[self.train_data[self.user_id] == user]
        user_items = list(user_data[self.item_id].unique())
        return user_items
    
    #Get unique users for a given item (song)
    def get_item_users(self, item):
        item_data = self.train_data[self.train_data[self.item_id] == item]
        item_users = set(item_data[self.user_id].unique())
        return item_users
        
    #Get unique items (songs) in the training data
    def get_all_items_train_data(self):
        all_items = list(self.train_data[self.item_id].unique())
        return all_items
        
    #Construct cooccurence matrix
    def construct_cooccurence_matrix(self, user_songs, all_songs):
        # Get users for all songs in user_songs
        user_songs_users = [self.get_item_users(song) for song in user_songs]
        
        # Initialize the item cooccurrence matrix
        cooccurence_matrix = np.zeros((len(user_songs), len(all_songs)))
        
        # Calculate cosine similarity between user songs and all unique songs in the training data
        for i, song in enumerate(all_songs):
            # Calculate unique listeners (users) of song (item) i
            songs_i_users = self.get_item_users(song)
            
            for j, user_song in enumerate(user_songs):
                # Get unique listeners (users) of user_song
                user_song_users = user_songs_users[j]
                
                # Calculate cosine similarity
                intersection = len(songs_i_users.intersection(user_song_users))
                norm = np.sqrt(len(songs_i_users) * len(user_song_users))
                if norm != 0:
                    cooccurence_matrix[j, i] = intersection / norm
                else:
                    cooccurence_matrix[j, i] = 0
        
        return cooccurence_matrix
    
    #Use the cooccurence matrix to make top recommendations
    def generate_top_recommendations(self, user, cooccurence_matrix, all_songs, user_songs):
        # Calculate a weighted average of the scores in cooccurence matrix for all user songs
        user_sim_scores = cooccurence_matrix.sum(axis=0) / cooccurence_matrix.shape[0]
        
        # Sort the indices of user_sim_scores based upon their value
        sort_index = np.argsort(user_sim_scores)[::-1]
        
        # Create a dataframe to store recommendations
        columns = ['user_id', 'song', 'score', 'rank']
        df = pd.DataFrame(columns=columns)
        
        # Fill the dataframe with top 10 item based recommendations
        rank = 1 
        for i in sort_index:
            if all_songs[i] not in user_songs and rank <= 10:
                df.loc[len(df)] = [user, all_songs[i], user_sim_scores[i], rank]
                rank += 1
        
        # Handle the case where there are no recommendations
        if df.shape[0] == 0:
            print("The current user has no songs for training the item similarity based recommendation model.")
            return -1
        else:
            return df
    
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

    def recommend(self, user):
        
        #A. Get all unique songs for this user
        user_songs = self.get_user_items(user)
        
        #B. Get all unique items (songs) in the training data
        all_songs = self.get_all_items_train_data()
        
        print("no. of unique songs in the training set: %d" % len(all_songs))
         
        #C. Construct item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        
        #D. Use the cooccurence matrix to make recommendations
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)
        
        return df_recommendations
    
    def get_similar_items(self, item_list):
        
        #A. assign item_list to user_songs
        user_songs = item_list
        
        #B. Get all unique items (songs) in the training data
        all_songs = self.get_all_items_train_data()
        
        print("no. of unique songs in the training set: %d" % len(all_songs))
         
        #C. Construct item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        
        #D. Use the cooccurence matrix to make recommendations
        df_recommendations = self.generate_top_recommendations("", cooccurence_matrix, all_songs, user_songs)
        
        return df_recommendations

In [17]:
ir = item_similarity_recommender_py()
ir.create(df, 'user_id', 'song')

In [18]:
user_items = ir.get_user_items(df['user_id'][5])

In [19]:
# display user songs history
for user_item in user_items:
    print(user_item)

The Cove - Jack Johnson
Entre Dos Aguas - Paco De Lucia
Stronger - Kanye West
Constellations - Jack Johnson
Learn To Fly - Foo Fighters
Apuesta Por El Rock 'N' Roll - Héroes del Silencio
Paper Gangsta - Lady GaGa
Stacked Actors - Foo Fighters
Sehr kosmisch - Harmonia
Heaven's gonna burn your eyes - Thievery Corporation feat. Emiliana Torrini
Let It Be Sung - Jack Johnson / Matt Costa / Zach Gill / Dan Lebowitz / Steve Adams
I'll Be Missing You (Featuring Faith Evans & 112)(Album Version) - Puff Daddy
Love Shack - The B-52's
Clarity - John Mayer
I?'m A Steady Rollin? Man - Robert Johnson
The Old Saloon - The Lonely Island
Behind The Sea [Live In Chicago] - Panic At The Disco
Champion - Kanye West
Breakout - Foo Fighters
Ragged Wood - Fleet Foxes
Mykonos - Fleet Foxes
Country Road - Jack Johnson / Paula Fuga
Oh No - Andrew Bird
Love Song For No One - John Mayer
Jewels And Gold - Angus & Julia Stone
Warning - Incubus
83 - John Mayer
Neon - John Mayer
The Middle - Jimmy Eat World
High and 

In [20]:
# give song recommendation for that user
ir.recommend(df['user_id'][5])

no. of unique songs in the training set: 9953


,user_id,song,score,rank
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Great Indoors - John Mayer,0.057321,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Times Like These - Jack Johnson,0.055596,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Bubble Toes - Jack Johnson,0.054856,3
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Breakdown - Jack Johnson,0.054502,4
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Quiet Houses - Fleet Foxes,0.054395,5
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Heard Them Stirring - Fleet Foxes,0.054115,6
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Belle - Jack Johnson,0.053498,7
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Posters - Jack Johnson,0.053468,8
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Meadowlarks - Fleet Foxes,0.053297,9
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,All At Once - Jack Johnson,0.052963,10


In [21]:
# give song recommendation for that user
ir.recommend(df['user_id'][111])

no. of unique songs in the training set: 9953


,user_id,song,score,rank
0,e006b1a48f466bf59feefed32bec6494495a4436,Pursuit Of Happiness (nightmare) - Kid Cudi / ...,0.156610,1
1,e006b1a48f466bf59feefed32bec6494495a4436,Lucky (Album Version) - Jason Mraz & Colbie Ca...,0.152002,2
2,e006b1a48f466bf59feefed32bec6494495a4436,Somebody To Love - Justin Bieber,0.150416,3
3,e006b1a48f466bf59feefed32bec6494495a4436,The Only Exception (Album Version) - Paramore,0.148908,4
4,e006b1a48f466bf59feefed32bec6494495a4436,Bleed It Out [Live At Milton Keynes] - Linkin ...,0.145361,5
5,e006b1a48f466bf59feefed32bec6494495a4436,The Scientist - Coldplay,0.140918,6
6,e006b1a48f466bf59feefed32bec6494495a4436,Love Story - Taylor Swift,0.134358,7
7,e006b1a48f466bf59feefed32bec6494495a4436,Billionaire [feat. Bruno Mars] (Explicit Albu...,0.133817,8
8,e006b1a48f466bf59feefed32bec6494495a4436,Bulletproof - La Roux,0.129525,9
9,e006b1a48f466bf59feefed32bec6494495a4436,Heartbreak Warfare - John Mayer,0.129125,10


# Content based Recommendation system

In [22]:
 def get_similar_items(self, item_list):
        
        #A. assign item_list to user_songs
        user_songs = item_list
        
        #B. Get all unique items (songs) in the training data
        all_songs = self.get_all_items_train_data()
        
        print("no. of unique songs in the training set: %d" % len(all_songs))
         
        #C. Construct item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        
        #D. Use the cooccurence matrix to make recommendations
        df_recommendations = self.generate_top_recommendations("", cooccurence_matrix, all_songs, user_songs)
        
        return df_recommendations

In [23]:
# give related songs based on the words
ir.get_similar_items(['Love Story - Taylor Swift', 'The Scientist - Coldplay'])

no. of unique songs in the training set: 9953


,user_id,song,score,rank
0,,Clocks - Coldplay,0.353573,1
1,,Fix You - Coldplay,0.316751,2
2,,Lucky (Album Version) - Jason Mraz & Colbie Ca...,0.313731,3
3,,Use Somebody - Kings Of Leon,0.299987,4
4,,The Only Exception (Album Version) - Paramore,0.298842,5
5,,Hey_ Soul Sister - Train,0.298154,6
6,,Fireflies - Charttraxx Karaoke,0.293886,7
7,,Marry Me - Train,0.291606,8
8,,Secrets - OneRepublic,0.290138,9
9,,Yellow - Coldplay,0.288546,10
